## My custom design notes
* For the end-caps for the rods use crush ribs. 
  * The hole should be drafted at about 2 degrees, the ribs should not be drafted. 
  * The ribs should interfer with the rod by .254 mm and that is the dimensions of the radius of it. 
  * The bottom of the draft hole is the same dimension as the rod. 

## Motor sizing calculations:
### Summary: 
Motor torque (combine torque of two wheels): 1.6 Nm  
RPM of wheels: 300

### Sympy printing function and setup

In [896]:
from sympy import *
from IPython.display import display, Math

def printEvaluation(expr, values, var_name, unit=''):
    """Printing funciton for sympy expressions to show full expression and calculation

    Parameters
    ----------
    expr
        A sympy expression to be displayed and evaluated
    values : dict
        A dictionary containing each of the varables in the expression as keys and their values as the ...values. 
        The varaible that is being calculated in this fuction gets added to values. 
    var_name : str
        Just the str version of the expression's name (the left side of the equation)
    unit : str
        (optional) The engineering units of the answer (e.g. N, kg, etc.)
    Returns
    -------
    The calculated value

    Example
    -------
    theta, y = symbols('theta, y')
    g = cos(theta / y)
    values = {theta: 1, y: 4}
    printEvaluation(g, values, 'g')
    """

    # Make an alternative dict with values that won't be evaluated. 
    uneval_vals = {}
    for key, value in values.items():
        uneval_vals[key] = UnevaluatedExpr(value)

    result = expr.evalf(subs=values)
    display(Math(f'{var_name} = \
                 {latex(nsimplify(expr))} = \
                    {latex(expr.subs(uneval_vals))} = \
                        {result:.4f} \\text{{ }} {unit}'))
    var_name = symbols(var_name)
    values[var_name] = result
    return var_name

### First reference calculations

In [897]:
# From: https://medium.com/husarion-blog/10-steps-to-choosing-the-right-motors-for-your-robotic-project-bf5c4b997407
# Inputs (everything in SI units): 

m_R, v_n, D_W, t_A, k, g = symbols("m_R, v_n, D_W, t_A, k, g")
values = {m_R: .907,  # Mass of the robot in kg
          v_n: 2.2,   # Linear speed of the robot in m/s
          D_W: 0.143, # Diameter of wheel in m
          t_A: 0.5,   # Time it takes to get to full speed in s 
          k:   1,  # Percent grade in %  
          g:   9.81}  # gravity in m/s^2

# .907 kg is 2 lbs
# 2.2 m/s is 5 mph
# 1 slope is 45 degrees

In [898]:
# Wheel rotation speed [RPM]
N_T = (60*v_n)/(pi*D_W)
N_T = printEvaluation(N_T, values, 'N_T', 'rpm')

<IPython.core.display.Math object>

In [899]:
# Thrust force for whole robot to get up a hill [N]
F_T = g*k*m_R
F_T = printEvaluation(F_T, values, 'F_T', 'N')

<IPython.core.display.Math object>

In [900]:
# Mechanical power for both motors together [W]
P_U = F_T*v_n
P_U = printEvaluation(P_U, values, 'P_U', 'W') 

<IPython.core.display.Math object>

In [901]:
# Torque per motor [Nm]
safety_factor = 2
T_T = (D_W/2)*F_T*safety_factor  # The example did per wheel but each of our motors drives 2 wheels. 
T_T = printEvaluation(T_T, values, 'T_T', 'Nm')

<IPython.core.display.Math object>

In [902]:
# Vehicle acceleration 
A_R = v_n / t_A
A_R = printEvaluation(A_R, values, 'A_R', 'm/s')

<IPython.core.display.Math object>

In [903]:
# Acceleration force
F_A = A_R*m_R
F_A = printEvaluation(F_A, values, 'F_A', 'N')
# Since this is less that F_T I'm not going to use it. 

<IPython.core.display.Math object>

### Second reference calculations (I think this one is wrong)

In [904]:
# From https://wiki.dfrobot.com/How_to_Calculate_the_Motor_Torque_for_a_Mobile_Robot


m, g, mu, v, t_A, r = symbols('m, g, mu, v, t_A, r')
values = {
    m:  0.907,  # mass in kg
    g:  9.8,    # gravity m/s^2
    mu: 0.7,    # friction (unitless)
    v: 2.2,     # desired speed m/s
    t_A: 0.5,   # time to get to speed
    r: 0.143/2, # radius of wheel in m
}

In [905]:
# Friction force: 
N = m * g
N = printEvaluation(N, values, 'N', 'N')
F_f = mu * N
F_f = printEvaluation(F_f, values, "F_f", 'N')
## I think this is wrong, seems like this assumes I'm dragging the body of the robot along the floor. 

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [906]:
# Acceleration force: 
a = v/t_A
a = printEvaluation(a, values, 'a', 'm/s^2')
F_a = m * a
F_a = printEvaluation(F_a, values, 'F_a', 'N')
F = F_f + F_a
F = printEvaluation(F, values, "F", "N")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [907]:
# Torque per motor
safety_factor = 2
tau = (F*r*safety_factor)/2  # Each motor is two wheel. 
tau = printEvaluation(tau, values, 'tau', 'Nm')

<IPython.core.display.Math object>

In [908]:
# Wheel rotation speed
n = (v*60)/(2*pi*r)
n = printEvaluation(n, values, 'n', 'rpm')

<IPython.core.display.Math object>

### Third reference calculations

In [909]:
# Ref: https://www.robotsforroboticists.com/motor-selection/

m_R, v_n, D_W, t_A, theta, eta, g = symbols("m_R, v_n, D_W, t_A, theta, eta, g")
values = {m_R:   0.907, # Mass of the robot in kg
          v_n:   2.2,   # Linear speed of the robot in m/s
          r:   0.143/2, # Diameter of wheel in m
          t_A:   0.5,   # Time it takes to get to full speed in s 
          theta: 0.785, # angle of slope  
          eta:   .8,    # efficiency
          g:     9.81}  # gravity in m/s^2

# .907 kg is 2 lbs
# 2.2 m/s is 5 mph
# .785 rad is 45 degrees

In [910]:
# Torque per motor
a = v_n/t_A
a = printEvaluation(a, values, 'a')
safety_factor = 2
T = (a*g*sin(theta)*m_R*r)/4*eta*safety_factor*2
T = printEvaluation(T, values, 'T', 'Nm')

<IPython.core.display.Math object>

<IPython.core.display.Math object>